Import necessary frameworks.

In [8]:
import math
import random
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf
from tensorflow import keras

import utils
import HINITE
import ourlayers
import evaluation
import os

Load the Youtube dataset.

In [2]:
data = utils.load_data("Flk")
train_idx, val_idx, test_idx = utils.split_train_val_test(data[0][0], 0.7, 0.15, 0.15)

Set parameters for the HINITE's architecture.

In [3]:
configs = utils.config_pare_HINITE(
    iterations=2000,
    lr_rate=0.001,
    lr_weigh_decay=0.001,
    flag_early_stop=True,
    use_batch=512,
    rep_alpha=[0.05,0.1,0.5,1.0,1.5],
    flag_norm_gnn=False,
    flag_norm_rep=False,
    out_dropout=0.2,
    GNN_dropout=0.2,
    rep_dropout=0.2,
    inp_dropout=0.0,
    rep_hidden_layer=3,
    rep_hidden_shape=[128, 64, 64],
    GNN_hidden_layer=3,
    GNN_hidden_shape=[64, 64, 32], 
    divide=True,
    head_num_att=1,
    out_T_layer=3,
    out_C_layer=3,
    out_hidden_shape=[128, 64, 32],
    GNN_alpha=[0.0],
    hete_att_size=[128, 128, 64]
)
activation = tf.nn.relu

Choose the model (with different hyperparameters) by checking $\epsilon_{\rm{ATE/PEHE}}$ in the validation set, and save the result in the test set.

In [ ]:
for i in range(0, 10):
    temp = [0.01, 0.1, 0.5, 1.0, 1.5]
    cur_all_input, yf, mu1, mu0, adjs= utils.data_preparation('Flk', i, data)

    true_ite = mu1 - mu0
    val_true_ite = true_ite[val_idx]
    print("val true ite", val_true_ite)
    val_true_ate = np.mean(true_ite[val_idx])
    test_true_ite = true_ite[test_idx]
    print("test ite", val_true_ite)
    test_true_ate = np.mean(true_ite[test_idx])

    cur_all_input = tf.cast(cur_all_input, tf.float32)
    val_true_ate = tf.cast(val_true_ate, tf.float32)
    test_true_ate = tf.cast(test_true_ate, tf.float32)

    best_pehe = 1e10
    best_err_ate = 1e10
    best_model = None
    for j in range(len(temp)):
        cur_path = "./save_Models_HINITE/Model_Flk_HINITE.HINITE_split_" + str(i) +"_rep_alpha" + str(temp[j]) 
        cur_name = "model"
        cur_model = utils.load_my_model(cur_path, cur_name, HINITE.HINITE, configs[0], tf.nn.relu, adjs)
        pehe_val, err_ate_val = evaluation.evalate(
            model=cur_model, 
            input_tensor=cur_all_input,
            test_idx=val_idx,
            true_ate=val_true_ate,
            true_y=yf,
            true_ite=val_true_ite
        )

        if err_ate_val < best_err_ate:
            best_err_ate  = err_ate_val
            print("best_err_ate", err_ate_val)
            best_model = cur_model
        elif err_ate_val == best_err_ate and pehe_val < best_pehe:
            best_pehe = pehe_val
            best_model = cur_model

        pehe_test, err_ate_test = evaluation.evalate(
            model=best_model, 
            input_tensor=cur_all_input, 
            test_idx=test_idx, 
            true_ate=test_true_ate,
            true_y=yf,
            true_ite=test_true_ite
        )
        cur_test_results = [pehe_test, err_ate_test]
        cur_test_results_name = './results/test_results_Flk_' + str(HINITE.HINITE)[8:-2]  +  "_split_" + str(i) 
        os.makedirs(os.path.dirname(cur_test_results_name), exist_ok=True)
        utils.save_results(cur_test_results, cur_test_results_name)

Load result that you saved.

In [ ]:
ate_all = []
pehe_all = []
for i in range(10):
    cur_test_results_name = './results/test_results_Flk_' + str(HINITE.HINITE)[8:-2]  +  "_split_" + str(i)  +".npy"
    cur_result = np.load(cur_test_results_name, allow_pickle=True)

    cur_ate =  cur_result[1]
    cur_pehe =  cur_result[0]

    ate_all.append(np.float(cur_ate))
    pehe_all.append(np.float(cur_pehe))

print(ate_all)
print(pehe_all)

Evaluate results on the Youtube dataset.

In [ ]:
# Flk
print(np.mean(pehe_all), np.std(pehe_all))
print(np.mean(ate_all), np.std(ate_all))